In [53]:
import pydeck as pdk
import pandas as pd
import numpy as np

from typing import List, Tuple
import geopandas as gpd
from copy import deepcopy
import h3
from shapely.geometry import Polygon
import os
import json 

from src.settings import DATA_DIR

from ipywidgets import HTML
from IPython.display import display

In [3]:
def get_hex_from_city_geojson(
    city: dict,
    resolution: int = 8
) -> Tuple[gpd.GeoDataFrame, List[str]]:
    """Fills city poolygon with hexagons at a given resolution.

    Args:
        city (dict): Geojson formatted city polygon
        resolution (int, optional): Uber H3 resolution. Defaults to 8.

    Returns:
        Tuple[gpd.GeoDataFrame, List[str]]: geodataframe with hexagons
        in city and list of hexagons ids.

    """
    city = deepcopy(city)
    city['geometry']['coordinates'][0] = [
        [x[1], x[0]] for x in city['geometry']['coordinates'][0]
    ]
    hexagons = h3.polyfill(city['geometry'], resolution)

    df = pd.DataFrame(hexagons, columns=['hex_id'])

    df['geometry_dict'] = df['hex_id'].apply(
        lambda x: {
            "type": "Polygon",
            "coordinates": [h3.h3_to_geo_boundary(h=x, geo_json=True)]
        })

    df['geometry'] = df['hex_id'].apply(
        lambda x: Polygon(h3.h3_to_geo_boundary(h=x, geo_json=True)))

    gdf = gpd.GeoDataFrame(df)

    return gdf, hexagons

In [37]:
with open(os.path.join(DATA_DIR, 'wroclaw.geojson')) as f:
    wroclaw_geojson = json.load(f)

wro_df, hex_wro = get_hex_from_city_geojson(wroclaw_geojson, resolution=8)

In [38]:
wro_df = wro_df.set_crs(epsg=4326)

In [39]:
gjson = wro_df.__geo_interface__

In [85]:
def hex_click(instance, payload):
    try:
        tmp = payload['data']['object']['hex_id']
        text.value = f'HEX ID: {tmp}'
    except Exception as e:
        text.value = 'Error: %s' % e

In [95]:
WRO_POS = [51.1239095, 17.0055833]
WRO_ZOOM = 11

text = HTML(value='Move the viewport')

layer = pdk.Layer(
    "GeoJsonLayer",
    data=wro_df,
    opacity=0.2,
    get_fill_color=[42, 111, 76],
    get_line_color=[0, 0, 0],
    line_width_min_pixels=2,
    pickable=True,
)


view_state = pdk.ViewState(
    longitude=WRO_POS[1],
    latitude=WRO_POS[0],
    zoom=WRO_ZOOM)


r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.deck_widget.on_click(hex_click)
display(text)
r.show()
# r.to_html('hexagon-example.html')

HTML(value='Move the viewport')

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…